In [56]:
import pandas as pd
import numpy as np

## Import Wrangled Datasets

In [85]:
elections = pd.read_csv("C:\\Users\\Roy\\Desktop\\Project\\ElectionData\\electionsFixed.csv")

age = pd.read_csv("C:\\Users\\Roy\\Desktop\\Project\\CountyPopData\\AgeGrpPopulations\\Age_Fixed.csv")
edu = pd.read_csv("C:\\Users\\Roy\\Desktop\\Project\\CountyPopData\\Education\\Education_Fixed.csv")
raceSex = pd.read_csv("C:\\Users\\Roy\\Desktop\\Project\\CountyPopData\\RaceSexPopulations\\RaceSexPops.csv")
unemployment = pd.read_csv("C:\\Users\\Roy\\Desktop\\Project\\CountyPopData\\Unemployment\\Unemployment_Fixed.csv")

## Merge Tables

In [86]:
del unemployment["County"] #Redundent info not needed for joining
raceSex = raceSex.rename(columns = {"state":"State FIPS Code", "county":"County FIPS Code", "year":"Year"})

finalDF = pd.merge(raceSex, unemployment, how = "inner",
                  on = ["State FIPS Code","County FIPS Code", "Year"])

In [87]:
del edu["FIPS Code"] #Delete redundent info again
del edu["State"]
del edu["County"]

finalDF = pd.merge(finalDF, edu, how = "inner",
                  on = ["State FIPS Code","County FIPS Code", "Year"])

In [88]:
del age["population"] #redundent info
age = age.rename(columns = {"year":"Year"})

finalDF = pd.merge(finalDF, age, how = "inner",
                  on = ["State FIPS Code","County FIPS Code", "Year"])

In [89]:
finalDF = finalDF.rename(columns = {"stname":"state","ctyname":"county", "Year":"year"})

finalDF = pd.merge(elections, finalDF, how = "inner",
                  on = ["state","county", "year"])

del finalDF["State FIPS Code"] #Only used FIPS for joining, not needed going forward
del finalDF["County FIPS Code"]

finalDF = finalDF.rename(columns = {"tot_pop":"population"})

In [102]:
#test = finalDF[(finalDF['office'] == 'Senate')&(finalDF['state'] == 'Alabama')&(finalDF['county'] == 'Autauga')]
#test[['year','LastPartyWon','LastDemVotePercent','LastRepVotePercent','LastPartyWon_2','LastDemVotePercent_2','LastRepVotePercent_2',
     #'LastPartyWon_3','LastDemVotePercent_3','LastRepVotePercent_3']]

In [93]:
finalDF['LastPartyWon'].value_counts() # value = 0 is from us (When no previous data)

R    49230
D    25942
Name: LastPartyWon, dtype: int64

In [92]:
#Drop < 300 rows that were not D/R outcome
dropList = ['I','IR','IS','AC','L','IND','ACP','RF','C']
finalDF = finalDF[~finalDF['LastPartyWon'].isin(dropList)]
finalDF = finalDF[~finalDF['winningParty'].isin(dropList)]

finalDF = finalDF[~finalDF['LastPartyWon_2'].isin(dropList)]
finalDF = finalDF[~finalDF['LastPartyWon_3'].isin(dropList)]

In [100]:
finalDF = finalDF[~((finalDF['LastDemVotePercent'] == 0) & (finalDF['LastRepVotePercent'] == 0))]
finalDF = finalDF[~((finalDF['LastDemVotePercent_2'] == 0) & (finalDF['LastRepVotePercent_2'] == 0))]

In [103]:
finalDF.to_csv("combinedData.csv", encoding = 'utf-8', index = False)